## N-gram 언어 모델로 문장 생성

네이버에서 오픈소스 제공 nsmc 영화 리뷰 데이터셋을 이용한다

In [3]:
import nltk
from nltk.util import ngrams
from nltk import word_tokenize
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist
import numpy as np
import codecs
from tqdm import tqdm
import random

In [4]:
# 한국어 처리에 필요한 konlpy 패키지를 설치하기 전에 선행 파일을 설치한다. 
from konlpy.tag import Okt
JAVA_HOME = "C:\Program Files\Java\jdk1.8.0_202"

In [5]:
# NLTK 패키지를 이용하여 입력 텍스트를 N-gram 형태로 변환한다.
sentence = "나는 매일 아침 아홉시 수업이 있다. 너무 힘들다"

In [6]:
# NLTK 사용을 위하여 선행 패키지를 설치한다.
nltk.download('punkt')

# 입력 텍스트를 띄어쓰기 기준으로 토큰화한다.
tokens = word_tokenize(sentence) 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hertaehoon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
print(tokens)

['나는', '매일', '아침', '아홉시', '수업이', '있다', '.', '너무', '힘들다']


In [8]:
# 한국어의 단어는 띄어쓰기를 기준으로 하지 않기 때문에 konlpy를 이용해 형태소를 기준으로 토큰화한다.
tagger = Okt()

def tokenize(text):
  tokens = ['/'.join(t) for t in tagger.pos(text)]
  return tokens

tokens = tokenize(sentence)
print(tokens)

['나/Noun', '는/Josa', '매일/Noun', '아침/Noun', '아홉시/Noun', '수업/Noun', '이/Josa', '있다/Adjective', './Punctuation', '너무/Adverb', '힘들다/Adjective']


In [9]:
bigram = ngrams(tokens, 2)
trigram = ngrams(tokens, 3)

In [10]:
print("bigram: ")
for b in bigram:
  print(b)

print("\ntrigram: ")
for t in trigram:
  print(t) 

bigram: 
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '아홉시/Noun')
('아홉시/Noun', '수업/Noun')
('수업/Noun', '이/Josa')
('이/Josa', '있다/Adjective')
('있다/Adjective', './Punctuation')
('./Punctuation', '너무/Adverb')
('너무/Adverb', '힘들다/Adjective')

trigram: 
('나/Noun', '는/Josa', '매일/Noun')
('는/Josa', '매일/Noun', '아침/Noun')
('매일/Noun', '아침/Noun', '아홉시/Noun')
('아침/Noun', '아홉시/Noun', '수업/Noun')
('아홉시/Noun', '수업/Noun', '이/Josa')
('수업/Noun', '이/Josa', '있다/Adjective')
('이/Josa', '있다/Adjective', './Punctuation')
('있다/Adjective', './Punctuation', '너무/Adverb')
('./Punctuation', '너무/Adverb', '힘들다/Adjective')


In [11]:
bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="", right_pad_symbol="")
print("bigrams with padding: ")
for b in bigram:
  print(b)

bigrams with padding: 
('', '나/Noun')
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')
('탄다/Verb', '')


In [12]:
# 문장 생성을 위하여 네이버 영화 리뷰 데이터셋을 다운로드한다.
!wget -nc -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

In [13]:
# 다운로드 받은 데이터셋을 읽고 인덱스와 라벨을 제외한 텍스트 부분만 가져온다.
# codecs 패키지는 대용량 파일을 조금씩 읽을 수 있게 해준다. 

with codecs.open("ratings_train.txt", encoding='utf-8') as f:
  data = [line.split('\t') for line in f.read().splitlines()] # \n 제외
  data = data[1:] # header 제외
print("데이터셋: ", data[:10])
docs = [row[1] for row in data] # 텍스트 부분만 가져옴
print("\n텍스트 데이터:", docs[:5])
print("\n문장 개수: ",len(docs)) # 총 15만개의 문장으로 이루어진 데이터셋임을 알 수 있다.

데이터셋:  [['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0'], ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'], ['6483659', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1'], ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'], ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'], ['9443947', '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', '0'], ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'], ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]

텍스트 데이터: ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']

문장 개수:  150000


In [14]:
# 토큰화한 텍스트 데이터의 bigram을 모두 리스트에 추가한다.
sentences = []
for d in tqdm(docs):
  tokens = tokenize(d)
  bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="", right_pad_symbol="")
  sentences += [t for t in bigram]





100%|█████████████████████████████████████████████████████████████████████████| 150000/150000 [02:54<00:00, 857.14it/s]


In [15]:
print(sentences[:10])

[('', '아/Exclamation'), ('아/Exclamation', '더빙/Noun'), ('더빙/Noun', '../Punctuation'), ('../Punctuation', '진짜/Noun'), ('진짜/Noun', '짜증나네요/Adjective'), ('짜증나네요/Adjective', '목소리/Noun'), ('목소리/Noun', ''), ('', '흠/Noun'), ('흠/Noun', '.../Punctuation'), ('.../Punctuation', '포스터/Noun')]


In [16]:
cfd = ConditionalFreqDist(sentences)
print(cfd[""].most_common(5))

[('정말/Noun', 2718), ('이/Noun', 2371), ('진짜/Noun', 2232), ('이/Determiner', 2115), ('영화/Noun', 2069)]


In [17]:
# 주어진 토큰(c) 다음에 가장 많이 등장하는 n개의 단어를 반환하는 함수를 만든다.
def most_common(c, n, pos=None):
  if pos is None:
    return cfd[tokenize(c)[0]].most_common(n)
  else:
    return cfd["/".join([c, pos])].most_common(n)

In [18]:
print(most_common("나", 10))

[('는/Josa', 831), ('의/Josa', 339), ('만/Josa', 213), ('에게/Josa', 148), ('에겐/Josa', 84), ('랑/Josa', 81), ('한테/Josa', 50), ('참/Verb', 45), ('이/Determiner', 44), ('와도/Josa', 43)]


In [19]:
# 단어별 등장 빈도를 기반으로 조건부 확률을 추정한다.
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [20]:
# “.” 다음에 “”가 올 확률을 출력한다.
print(cpd[tokenize(".")[0]].prob(""))

0.39102658679807606


In [21]:
# 토큰 c 다음에 토큰 w가 bigram으로 함께 등장할 확률을 구한다.
def bigram_prob(c, w):
  context = tokenize(c)[0]
  word = tokenize(w)[0]
  return cpd[context].prob(word)

In [22]:
print(bigram_prob("이", "영화"))

0.4010748656417948


In [23]:
print(bigram_prob("영화", "이"))

0.00015767585785521414


In [24]:
# 조건부 확률을 알게 되면 가장 확률이 높은 토큰열을 토대로 문장을 생성할 수 있다.
def generate_sentence(seed=None, debug=False):
  if seed is not None:
    import random
    random.seed(seed)
  c = ""
  sentence = []
  while True:
    if c not in cpd:
      break
    w = cpd[c].generate()

    if w == "":
      break
    
    word = w.split("/")[0]
    pos = w.split("/")[1]

    # 조사, 어미 등을 제외하고 각 토큰은 띄어쓰기로 구분하여 생성한다.
    if c == "":
      sentence.append(word.title())
    elif c in ["`", "\"","'","("]:
      sentence.append(word)
    elif word in ["'", ".", ",", ")", ":", ";", "?"]:
      sentence.append(word)
    elif pos in ["Josa", "Punctuation", "Suffix"]:
        sentence.append(word)
    elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
                "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
        sentence.append(word)
    else:
        sentence.append(" " + word)
    c = w

    if debug:
      print(w)

  return "".join(sentence)

In [25]:
print(generate_sentence(2))

도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 합성 한 가수 노래와 흥행 놓친 영화다. 사투리 연기 하나 없는 ‘ 스피드 감 넘치는 스릴 넘치는 연기를 이해 되지 못 하시는 분보다 훨 재밌구만 평점을 망처 놓은 듯하다. 영화 보는이로 하여금 불편함을 느꼇을듯


In [26]:
generate_sentence(2, debug=True)

도리/Noun
까지/Josa
본/Verb
영화/Noun
너무/Adverb
.../Punctuation
뭔가/Noun
../Punctuation
최고/Noun
네/Suffix
요/Josa
./Punctuation
하지만/Conjunction
../Punctuation
눈물/Noun
낫다는건/Verb
또/Noun
영화/Noun
에/Josa
들지/Verb
않는다/Verb
./Punctuation
근데/Adverb
뭐/Noun
야/Josa
어떻게/Adjective
그렇게/Adverb
착했던/Adjective
윤재/Noun
랑은/Josa
에바/Noun
그린/Noun
드레스/Noun
소리/Noun
듣는거/Verb
임/Noun
"""/Punctuation
에리/Noun
욧의/Noun
미모/Noun
로/Josa
합성/Noun
한/Determiner
가수/Noun
노래/Noun
와/Josa
흥행/Noun
놓친/Verb
영화/Noun
다/Josa
./Punctuation
사투리/Noun
연기/Noun
하나/Noun
없는/Adjective
‘/Foreign
스피드/Noun
감/Noun
넘치는/Adjective
스릴/Noun
넘치는/Adjective
연기/Noun
를/Josa
이해/Noun
되지/Verb
못/VerbPrefix
하시는/Verb
분/Noun
보다/Josa
훨/Noun
재밌구만/Adjective
평점/Noun
을/Josa
망처/Noun
놓은/Verb
듯/Noun
하다/Verb
./Punctuation
영화/Noun
보는이로/Verb
하여금/Adverb
불편함을/Adjective
느꼇을듯/Noun


'도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 합성 한 가수 노래와 흥행 놓친 영화다. 사투리 연기 하나 없는 ‘ 스피드 감 넘치는 스릴 넘치는 연기를 이해 되지 못 하시는 분보다 훨 재밌구만 평점을 망처 놓은 듯하다. 영화 보는이로 하여금 불편함을 느꼇을듯'